In [1]:
import pymongo
cli = pymongo.MongoClient()
db = cli.test
coll = db.stress
coll.drop()

In [2]:
import time
from functools import wraps

def timed(label):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            n = kwargs.get('n')
            bs = kwargs.get('bs', 1)
            begin = time.time()
            try:
                return func(*args, **kwargs)
            finally:
                elapsed = time.time() - begin
                if n:
                    if bs:
                        n *= bs
                    print('{} {} / s'.format(
                        n / elapsed,
                        label))
                else:
                    print('{}: {}s elapsed'.format(
                        label, elapsed))
        return wrapper
    return decorator

In [3]:
@timed('inserts')
def insert_test(coll, n=1000, bs=1000):
    for ix_n in range(n):
        block = [
            {'ix_n': ix_n, 'ix_b': ix_b}
            for ix_b in range(bs)]
        coll.insert_many(block)
        if ix_n % 40 == 0:
            print()
        print('.', end='', flush=True)
    print()

In [4]:
import random
@timed('queries')
def query_test(coll, max_ix_b, max_ix_n, n=100, bs=10):
    for ix_n in range(n):
        rv_n = random.randint(0, max_ix_n + 1)
        rv_b = random.randint(0, max_ix_b + 1)
        res = coll.find({
            'ix_n': rv_n,
            'ix_b': {'$gt': rv_b}
        })
        list(res)


In [5]:
@timed('updates')
def update_test(coll, max_ix_b, max_ix_n, n=10000):
    for ix_n in range(n):
        rv_n = random.randint(0, max_ix_n + 1)
        rv_b = random.randint(0, max_ix_b + 1)
        coll.update_one(
            {'ix_n': rv_n, 'ix_b': rv_b},
            {'$inc': {'x': 1}})

In [6]:
insert_test(coll, n=500, bs=1000)


........................................
........................................
........................................
........................................
........................................
........................................
........................................
........................................
........................................
........................................
........................................
........................................
....................
59392.31088162727 inserts / s


In [7]:
max_ix_b = 999
max_ix_n = 499

In [8]:
coll.drop_indexes()

In [10]:
query_test(coll, max_ix_b, max_ix_n, n=50)

5.714430053619922 queries / s


In [11]:
update_test(coll, max_ix_b, max_ix_n, n=50)

9.950288696404058 updates / s


In [12]:
coll.create_index([
    ('ix_n', 1)
])

'ix_n_1'

In [13]:
query_test(coll, max_ix_b, max_ix_n, n=2000)

250.37396676738405 queries / s


In [14]:
update_test(coll, max_ix_b, max_ix_n, n=2000)

833.3267603174081 updates / s


In [15]:
coll.create_index([
    ('ix_n', 1),
    ('ix_b', 1)
])

'ix_n_1_ix_b_1'

In [16]:
coll.drop_index('ix_n_1')

In [17]:
query_test(coll, max_ix_b, max_ix_n, n=5000)

267.92104758862985 queries / s


In [18]:
update_test(coll, max_ix_b, max_ix_n, n=5000)

1399.9104843158975 updates / s


In [19]:
insert_test(coll, n=200, bs=1000)


........................................
........................................
........................................
........................................
........................................
50018.69602458299 inserts / s


In [20]:
from threading import Thread
threads = [
    Thread(target=insert_test, args=(coll,), kwargs={'n': 200, 'bs': 1000}),
    Thread(target=update_test, args=(coll,), 
           kwargs={'n': 2000, 'max_ix_n': max_ix_n, 'max_ix_b': max_ix_b}),
    Thread(target=query_test, args=(coll,), 
           kwargs={'n': 2000, 'max_ix_n': max_ix_n, 'max_ix_b': max_ix_b})]
for t in threads:
    t.setDaemon(True)
    t.start()
for t in threads:
    t.join()


........................................
........................................
........................................
........................................
........................................
38081.11597728409 inserts / s
318.695173861869 updates / s
143.74300175424693 queries / s
